In [ ]:
# populate env variable os.environ.get("REACTOR_MODELS_DIR") to current directory
import os
os.environ["REACTOR_MODELS_DIR"] = '.'
os.environ['REACTOR_FACERESTORE_MODELS_PATH'] = '.'


from nodes import reactor
from nodes import RestoreFace



In [ ]:
reactorobj = reactor()
from PIL import Image
input_image = Image.open("/Users/abhinavg/Downloads/ComfyUI_00204_.png").convert("RGB")
source_image = Image.open("/Users/abhinavg/Downloads/Gopal_Abhinav_594-PilotSF-1428.jpg").convert("RGB")
import torchvision.transforms as transforms 
transform = transforms.Compose([ 
    transforms.PILToTensor() 
]) 
  
input_image = transform(input_image).unsqueeze(0)/255
source_image = transform(source_image).unsqueeze(0)/255
res=  reactorobj.execute(True, input_image, "inswapper_128.onnx", 0, "no", "0", "0", 1, "codeformer.pth", 1, 1, "YOLOv5n", source_image, None )

import numpy as np
from PIL import Image

img_array = np.uint8(res[0].squeeze()*255)
# change img_array to RGB from BGR
Image.fromarray(img_array)

In [ ]:
import numpy as np
import cv2

img_array = np.uint8(res[0].squeeze()*255)
from PIL import Image
image = Image.fromarray(img_array)

In [ ]:
# takes in a path to an image that is the MiDas output

import cv2
import numpy as np
midas_img_path = "/Users/abhinavg/Downloads/temp.png"

from PIL import Image
import numpy as np
import torch
def image_to_mask(image, channel):
    channels = ["red", "green", "blue", "alpha"]
    mask = image[:, :, :, channels.index(channel)]
    return (mask,)

def invert(mask):
    out = 255 - mask
    return (out,)

def inpaint_preprocessor(image, mask):
    # Resize the mask using cv2.resize
    resized_mask = cv2.resize(mask, (image.shape[1], image.shape[0]), interpolation=cv2.INTER_LINEAR)
    # Expand the mask to have three channels
    expanded_mask = np.tile(resized_mask[..., np.newaxis], (1, 1, 3))/255.0

    # Clone the image
    image_clone = np.copy(image)

    # Set masked pixels to -1.0
    image_clone[expanded_mask > 0.5] = -1.0

    return (image_clone,)

# read in rgba image
midas_image = cv2.imread(midas_img_path, cv2.IMREAD_UNCHANGED)
midas_image_tensor = np.array(midas_image).reshape((1, midas_image.shape[0], midas_image.shape[1], midas_image.shape[2]))
midas_mask = image_to_mask(midas_image_tensor, "red")
midas_mask = np.squeeze(midas_mask)
cv2.imwrite("midas_mask.png", midas_mask)
inverted_mask = invert(midas_mask)
inverted_mask = np.squeeze(inverted_mask)
cv2.imwrite("inverted_midas_mask.png", inverted_mask)


orig_img_path = "/Users/abhinavg/Downloads/jerm.png"
orig_image = cv2.imread(orig_img_path)
orig_image_expanded = np.array(orig_image).reshape((1, orig_image.shape[0], orig_image.shape[1], orig_image.shape[2]))

inpaint_preprocessed = inpaint_preprocessor(orig_image, inverted_mask)
cv2.imwrite("inpaint_preprocessed.png", inpaint_preprocessed[0])

In [ ]:
from diffusers import DiffusionPipeline
import torch

pipe_id = "stabilityai/stable-diffusion-xl-base-1.0"
pipe = DiffusionPipeline.from_pretrained(pipe_id).to("cpu")

In [ ]:
pipe.load_lora_weights("CiroN2022/toy-face", weight_name="toy_face_sdxl.safetensors", adapter_name="toy")

In [ ]:
prompt = "toy_face of a hacker with a hoodie"

lora_scale= 0.9
image = pipe(
    prompt, num_inference_steps=30, cross_attention_kwargs={"scale": lora_scale}, generator=torch.manual_seed(0)
).images[0]
image